In [1]:
import pandas as pd
import numpy as np
import json
import tqdm

# preprocessing yelp

In [2]:
cols = ['business_id', 'text']
data = []
file_name = './dataset-downloaded-data/yelp/yelp_academic_dataset_review.json'
cnt = 0 
with open(file_name, encoding='latin-1') as f:
    for line in tqdm.tqdm_notebook(f):
        doc = json.loads(line)
        lst = [doc['business_id'], doc['text']]
        data.append(lst)

/tmp/ipykernel_92129/1815763231.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm.tqdm_notebook(f):


0it [00:00, ?it/s]

In [3]:
df = pd.DataFrame(data=data, columns=cols)

In [4]:
bdf = pd.read_json('./dataset-downloaded-data/yelp/yelp_academic_dataset_business.json', lines=True)
bdf = bdf[bdf.categories.isna()==False]

In [6]:
bdf['cat0'] = bdf['categories'].str.split(', ').apply(lambda x: x[0])

In [7]:
temp = bdf.categories.str.split(', ').explode().value_counts().reset_index()
a = temp[temp['index'].str.lower().str.contains('salons')
     |
     temp['index'].str.lower().str.contains('beauty')
    ]['index'].values[:4]
print(a)

beauty = bdf[bdf.cat0.isin(a)]

beauty = beauty[['business_id','categories']]

beauty['topic'] = 'Beauty'

beauty.columns = ['business_id','origin_topic', 'topic']

['Beauty & Spas' 'Hair Salons' 'Nail Salons' 'Cosmetics & Beauty Supply']


In [8]:
temp[temp['index'].str.lower().str.contains('travel')]
a = ['Hotels & Travel']
travel = bdf[bdf.cat0.isin(a)]

travel = travel[['business_id','categories']]

travel['topic'] = 'Travel'

travel.columns = ['business_id','origin_topic', 'topic']

In [ ]:
# a = ['Automotive', 'Auto Repair', 'Auto Parts & Supplies',
#        'Auto Detailing']
# auto = bdf[bdf.cat0.isin(a)]

# auto = auto[['business_id','categories']]

# auto['topic'] = 'PersonalTransport'

# auto.columns = ['business_id','origin_topic', 'topic']

In [9]:
temp[temp['index'].str.lower().str.contains('transport')]

a = ['Public Transportation','Transportation']

transport = bdf[bdf.cat0.isin(a)]

transport = transport[['business_id','categories']]

transport['topic'] = 'MassTransit'

transport.columns = ['business_id','origin_topic', 'topic']

In [10]:
medical = bdf[bdf['cat0'].isin(['Medical Centers','Health & Medical'])]

medical = medical[['business_id','categories']]

medical['topic'] = 'Health&Medicine'

medical.columns = ['business_id','origin_topic', 'topic']

In [11]:
a = ['Nightlife', #'Bars', #'Sushi Bars',
     'Sports Bars', #'Juice Bars & Smoothies',
     'Wine Bars', 'Cocktail Bars', 'Whiskey Bars','Champagne Bars', 'Cigar Bars']
leisure = bdf[bdf['cat0'].isin(a)]

leisure = leisure[['business_id','categories']]

leisure['topic'] = 'Leisure'

leisure.columns = ['business_id','origin_topic', 'topic']

In [12]:
bid = pd.concat([beauty,travel, transport, medical, leisure])

In [13]:
result = df.merge(bid, on='business_id')

In [14]:
result = result[['text', 'topic', 'origin_topic']]

In [16]:
result.to_csv('./dataset-prepared-data/yelp_5topics.csv', index=False)

In [125]:
! free -h

              total        used        free      shared  buff/cache   available
Mem:           47Gi        28Gi        11Gi        12Mi       6.8Gi        18Gi
Swap:         6.0Gi       6.0Gi       2.0Mi


In [37]:
import IPython

In [39]:
IPython.display.Audio("../alarm_sound/What is love.mp3", autoplay=True)

# preproccess multiwoz

In [3]:
def utterances_from_multiwoz(column):
        res_df = pd.DataFrame()
        for i in column:
                df = pd.DataFrame([i])[:1]
                res_df = pd.concat([res_df,df])

        return res_df

def multiwoz_to_df(path):

    import tqdm
    import warnings
    warnings.filterwarnings("ignore")
    import gc
    import pandas as pd
    import os
    import json

    dirs = os.listdir(path)

    multiwoz = pd.DataFrame()
    for i in dirs:
        multiwoz= pd.concat([multiwoz, pd.read_json(path+i)])

    multiwoz.turns = multiwoz.turns.apply(utterances_from_multiwoz)

    res_df = pd.DataFrame()
    for i in tqdm.tqdm_notebook(multiwoz.values):
        df = i[2]
        df = df[df['speaker']!='SYSTEM']
        df['dialog_id'] = i[0]
        df['intent'] = str(i[1])

        res_df = pd.concat([res_df,df])
    del multiwoz
    gc.collect()
    
    return res_df

path1 = './downloaded-data/multiwoz/data/MultiWOZ_2.2/train/'
res_df1 = multiwoz_to_df(path1)

path2 = './downloaded-data/multiwoz/data/MultiWOZ_2.2/test/'
res_df2 = multiwoz_to_df(path2)


df = pd.concat([res_df2, res_df1])

df = df[df.turn_id=='0']

df = df[['dialog_id', 'utterance', 'intent']]

df.columns = ['dialog_id', 'text', 'intents']

df.intents = df.intents.apply(eval)

df = df.explode('intents')

df.intents = df.intents.astype('str')

mapper = {'restaurant':'food',
        'train':'MassTransit',
        'hotel':'Leisure',
        'taxi':'MassTransit',
        'attraction':'Leisure',
        'nan':np.nan,
        'hospital':'Health&Medicine',
        'bus':'MassTransit'}

df['topic'] = df['intents'].map(mapper)

p1 = df.groupby('text')['topic'].apply(list).reset_index()

p2 = df.groupby('text')['intents'].apply(list).reset_index()

p3 = df[['dialog_id','text']].drop_duplicates()

df = (
    p3
    .merge(p1, on='text')
    .merge(p2, on='text'))

df = df.iloc[:,1:]

df.columns = ['text','topic', 'origin_topic']

In [23]:
df.to_csv('./prepared-data/multiwoz.csv', index=False)

# preprocess ccpe 

In [24]:
df = pd.read_json('./downloaded-data/ccpe/data.json')

df.utterances = df.utterances.apply(pd.DataFrame)

ccpe = pd.DataFrame()
for i in df.values:
    l = i[1]
    l['dialog_id'] = i[0]
    ccpe = pd.concat([ccpe,l])

def segments_preprocess(col):
    if isinstance(col, list):
        intents = list()
        for i in col:
            intents.append(i['annotations'][0]['entityType'])
        intents = np.unique(intents)
    else:
        intents=None
    return intents

ccpe['intents'] = ccpe.segments.apply(segments_preprocess)


ccpe['topic'] = 'movies&tv'
ccpe = ccpe[['text','topic', 'intents']]
ccpe.columns = ['text','topic', 'origin_topic']

In [26]:
ccpe.to_csv('./prepared-data/ccpe.csv', index=False)

# preproccess reddit

In [225]:
df = pd.read_csv('./prepared-data/reddit-data.csv')

df = df.rename({'subreddit':'origin_topic'}, axis=1)

mapper = pd.read_csv('./input-data/DP Datsets - Классы и датасеты.csv')

mapper = mapper[mapper['Название датасета']=='Reddit']

mapper['Ссылка на датасет'] = mapper['Ссылка на датасет'].apply(lambda x: x.split('/')[-2])

mapper = mapper[['Название класса', 'Ссылка на датасет']]

mapper.columns = ['topic','origin_topic']

df = df.merge(mapper, on='origin_topic')

df['text'] = df['title']+ '.    ' + df['body']

df = df.drop(['title', 'body'], axis=1)

df.comments = df.comments.apply(lambda x: eval(x) if isinstance(x, str) else '')

title_body = df[['text','topic','origin_topic']]

title_body = title_body.drop_duplicates()

title_body = title_body.dropna()

comments = df[['comments', 'topic', 'origin_topic']].explode('comments')

comments = comments.dropna()

comments = comments.rename({'comments':'text'},axis=1)

In [228]:
comments.to_csv('./prepared-data/comments-reddit.csv', index=False)

title_body.to_csv('./prepared-data/title-body-reddit.csv', index=False)

# preprocessing self dialogue

In [6]:
import os

In [7]:
path = ('./downloaded-data/self_dialogue_corpus/corpus/')

selfd = pd.DataFrame()
for i in os.listdir(path):
    for j in os.listdir(path+i):
        j = pd.read_csv(path+i+'/'+j)
        j['intent'] = i
        selfd=pd.concat([selfd, j])

l = selfd.iloc[:,28:]

l = l.drop(['Approve','Reject'], axis=1)

cols = list(l.columns)

cols.remove('intent')

cols = sorted(cols, key=lambda x: int(x.split('sentence')[-1]))

l = l.fillna('')

l['all_cols'] = ''
for i in cols:
    l['all_cols']+= '@@'+l[i]
    
l = l[['all_cols', 'intent']]

l.all_cols = l.all_cols.str.split('@@')

l = l.explode('all_cols')

l = l[l.all_cols!='']

mapper = pd.read_csv('./input-data/DP Datsets - Классы и датасеты.csv')

mapper = mapper[mapper['Название датасета']=='The Self-dialogue Corpus']

mapper = mapper[['Топик', 'Название класса']]
mapper.columns = ['topic','intent']

l = l.merge(mapper, on='intent')

l = l[['all_cols','topic','intent']]

l.columns = ['text', 'topic', 'origin_topic']

In [246]:
l.to_csv('./prepared-data/self-dialogue.csv', index=False)

# preprocessing amazon

In [43]:
import pandas as pd
import gzip
import os
import tqdm

def parse(path):
    g = gzip.open(path, 'rb')
    true = True
    false = False
    for l in g:
        yield eval(l)

def getDF(path,name):
    i = 0
    df = {}
    for d in tqdm.tqdm_notebook(parse(path)):
        if i >300_000:
            break
        try:
            df[i] = d['reviewText']
            i += 1
        except:
            continue

    res = pd.DataFrame.from_dict(df, orient='index', columns=['text'])
    res['origin_topic'] = name
    
    return res

path = './downloaded-data/'
dirs = [i for i in os.listdir(path) if '.gz' in i]

res_df = pd.DataFrame()
for i in tqdm.tqdm_notebook(dirs):
    df = getDF(path+i,
          ' '.join(i.replace('.json.gz','').split('_')[:-1]))
    res_df = pd.concat([res_df,df])

mapper = pd.read_csv('./input-data/DP Datsets - Классы и датасеты.csv')

mapper = mapper[mapper['Название датасета']=='Amazon Review Data (2018)']

mapper = mapper[['Топик', 'Название класса']]
mapper.columns = ['topic','origin_topic']

res_df = res_df.merge(mapper, on='origin_topic')

In [66]:
res_df = res_df[res_df.topic!='no label']

In [67]:
%%time
res_df.to_csv('./prepared-data/amazon-reviews.csv', index=False)

CPU times: user 46.6 s, sys: 3.28 s, total: 49.9 s
Wall time: 49.9 s


# medium

In [30]:
import pandas as pd
import tqdm
import warnings
warnings.filterwarnings("ignore")

In [33]:
df = pd.read_csv('./downloaded-data/Medium_Clean.csv')
cols = list(df.iloc[:,13:].columns)

df = df[['Title','Subtitle']+cols]

In [36]:
for i in tqdm.tqdm_notebook(cols):
    df[i] = df[i].apply(lambda x: i if x==1 else '')

  0%|          | 0/95 [00:00<?, ?it/s]

In [38]:
df['all_cols'] = ''
for i in tqdm.tqdm_notebook(cols):
    df['all_cols']+= "@@"+df[i]
    df = df.drop([i],axis=1)

  0%|          | 0/95 [00:00<?, ?it/s]

In [42]:
df.all_cols = df.all_cols.str.split('@@').apply(lambda x: [i for i in x if i!=''])

df = df[df.all_cols.astype(str) != '[]']

df = df.explode('all_cols')

df.all_cols = df.all_cols.str.replace('Tag_','')

# CIRCA

In [7]:
df = pd.read_csv('./dataset-downloaded-data/circa/circa-data.tsv', sep='\t')

classes_base = pd.read_csv('./first-statistics/DP Datsets - Классы и датасеты.csv')

mapper = {'X wants to know what sorts of books Y likes to read.' : 'books&literature',
"X wants to know about Y's food preferences.": 'food',
'X wants to know what activities Y likes to do during weekends.':'leisure',
"X wants to know about Y's music preferences.":'music',
'Y has just told X that he/she is thinking of buying a flat in New York.':'no label',
'Y has just told X that he/she is considering switching his/her job.':'no label',
'X and Y are colleagues who are leaving work on a Friday at the same time.':'no label',
'X and Y are childhood neighbours who unexpectedly run into each other at a cafe.':'no label',
'Y has just moved into a neighbourhood and meets his/her new neighbour X.':'no label',
'Y has just travelled from a different city to meet X.':'no label'}

df['topic'] = df.context.map(mapper)

df = df[df.topic!='no label']

df = df[['question-X','topic', 'context']]

df.columns = ['text', 'topic', 'origin_topic']

df.to_csv('./dataset-prepared-data/circa.csv', index=False)

# depression

In [32]:
df = pd.read_csv('./dataset-downloaded-data/depression/Suicide_Detection.csv')

df = df[['text','class']]

df = df[df['class']!='non-suicide']

df['topic'] = 'depression'

df = df[['text','topic','class']]

df.columns = ['text', 'topic', 'origin_topic']

In [46]:
df.to_csv('./dataset-prepared-data/depression.csv', index=False)

In [47]:
df

,text,topic,origin_topic
0,Ex Wife Threatening SuicideRecently I left my ...,depression,suicide
3,i need helpjust help me im crying so hard,depression,suicide
4,"I’m so lostHello, my name is Adam (16) and I’v...",depression,suicide
5,Honetly idkI dont know what im even doing here...,depression,suicide
6,[Trigger warning] Excuse for self inflicted bu...,depression,suicide
...,...,...,...
232061,I’ve fucking had enough.None of this depressio...,depression,suicide
232062,paralyzing anxiety &amp; depression are pushin...,depression,suicide
232066,My mom tried to kill herself today. I feel los...,depression,suicide
232067,I’m a fucking scummy person and I can’t change...,depression,suicide


# women clothes

In [48]:
df = pd.read_csv('./dataset-downloaded-data/women-clothes/Womens Clothing E-Commerce Reviews.csv')

In [51]:
df['topic']= 'clothes'
df = df[['Review Text','topic','Class Name']]
df.columns = ['text', 'topic', 'origin_topic']

In [54]:
df.to_csv('./dataset-prepared-data/women-clothes.csv', index=False)

# yelp

In [4]:
category = pd.read_json('./dataset-downloaded-data/yelp/yelp_academic_dataset_business.json', lines=True)

In [12]:
%%time
df = pd.read_json('./dataset-downloaded-data/yelp/yelp_academic_dataset_review.json', lines=True, nrows=5_000_000)

CPU times: user 51.6 s, sys: 19.5 s, total: 1min 11s
Wall time: 1min 19s


In [15]:
df1 = df[['business_id','text']]

In [17]:
category = category[['business_id','categories']]

In [19]:
df1 = df1.merge(category, on='business_id')

In [22]:
k = df1.categories.str.split(', ').explode().value_counts().reset_index()

# reddit celebs

In [150]:
cr = pd.read_csv('./dataset-downloaded-data/celebs-reddit.csv')

In [160]:
p1 = cr[['title', 'subreddit']]
p1 = p1.rename({'title':'text'}, axis=1)
p2 = cr[['body', 'subreddit']]
p2 = p2.rename({'body':'text'}, axis=1)
tb = pd.concat([p1,p2])

tb = tb.dropna()

In [174]:
cm = cr[cr.comments.isna()==False]

cm['comments'] = cm['comments'].apply(eval)
cm = cm.explode('comments')

/tmp/ipykernel_91758/544051038.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cm['comments'] = cm['comments'].apply(eval)


In [176]:
cm = cm[['comments', 'subreddit']]

In [177]:
cm.columns = ['text','origin_topic']

In [182]:
tb = tb.rename({'subreddit': 'origin_topic'}, axis=1)

In [183]:
tbcm = pd.concat([tb,cm])

In [184]:
tbcm['topic'] = 'Celebrities&Events'

In [186]:
tbcm = tbcm[['text', 'topic', 'origin_topic']]

In [188]:
tbcm.to_csv('./dataset-prepared-data/celebs-reddit.csv', index=False)